[View in Colaboratory](https://colab.research.google.com/github/pamsfwang/perceptual_learning/blob/master/batch_stimuli.ipynb)

In [0]:
import tensorflow as tf
import numpy
import os
import pandas as pd
import matplotlib.pyplot as plt


In [6]:
#Mount Google drive on Colab
#step01
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse


from google.colab import auth
auth.authenticate_user()

from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()

import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}
#step02: code for mounting google drive on Linux
!mkdir -p drive
!google-drive-ocamlfuse drive

#step03: change directory to the google drive folder for the project
os.getcwd()
!ls
os.chdir('./drive/Colab_files')
os.getcwd()
!ls

gpg: keybox '/tmp/tmpe2b8869i/pubring.gpg' created
gpg: /tmp/tmpe2b8869i/trustdb.gpg: trustdb created
gpg: key AD5F235DF639B041: public key "Launchpad PPA for Alessandro Strada" imported
gpg: Total number processed: 1
gpg:               imported: 1
··········
fuse: mountpoint is not empty
fuse: if you are sure this is safe, use the 'nonempty' mount option
drive			     notebooks
jfj_stimuli_param.csv	     stimuli_param_uniqueF3.csv
jfj_stimuli_param_test.csv   stimuli_param_uniqueF3_test_new.csv
jfj_stimuli_param_train.csv  stimuli_param_uniqueF3_train_new.csv
models			     stimuli_param_uniqueF3_train_new.csv.ods
drive			     notebooks
jfj_stimuli_param.csv	     stimuli_param_uniqueF3.csv
jfj_stimuli_param_test.csv   stimuli_param_uniqueF3_test_new.csv
jfj_stimuli_param_train.csv  stimuli_param_uniqueF3_train_new.csv
models			     stimuli_param_uniqueF3_train_new.csv.ods


In [0]:
#specify variables
train_data = 'stimuli_param_uniqueF3_train_new.csv' #'jfj_stimuli_param_train.csv'
test_data = 'jfj_stimuli_param_test.csv'

num_features = 3
node_hidden01 = 10
learning_rate = 0.005
initial_weight01 = -0.001
initial_weight02 = 0.001

In [9]:
#load data
df=pd.read_csv(train_data, sep=',',header=None)
df.shape
nsamples =df.shape[0]
print "number of training samples:",nsamples

#shuffle the data
df_array = df.values
new = numpy.take(df_array,numpy.random.permutation(df_array.shape[0]),axis=0,out=df_array);

##separate data into features and labels (numpy.array)
temp = new[:,0:3] #df.iloc[:,0:num_features]
x_data = temp #temp.values
print "dimension of feature dataset:",x_data.shape
print(x_data[0:5,:])

temp = new[:,3]#df.iloc[:,num_features]
y_data = temp #temp.values
print "dimension of category dataset:",y_data.shape
print(y_data[0:5])

number of training samples: 52
dimension of feature dataset: (52, 3)
[[1.  3.  3.1]
 [3.  3.  4.4]
 [2.  3.  3.4]
 [4.  1.  1.7]
 [1.  2.  0.3]]
dimension of category dataset: (52,)
[1. 0. 0. 1. 1.]


In [0]:
##set up network layers
#You can think of most the tensorflow variables as actually being functions that we will call when we want to get their value. 
input_ph = tf.placeholder(tf.float32, shape=[num_features, None]) 
#This will be the place the input to the network is inserted
#shape: number of features
#None: take in any size

target_ph =  tf.placeholder(tf.float32, shape=[1,None]) 
#This will be the place the target for the network is insertedd
#shape = number of node

#First layer weights
W1 = tf.Variable(tf.random_uniform([node_hidden01,num_features],initial_weight01,initial_weight02)) 
#first layer: 3 input feautres to 10 nodes in the hidden layer
#W1: weights from item input to hidden layer 01
b1 = tf.Variable(tf.random_uniform([node_hidden01,1],initial_weight01,initial_weight02)) # " " biases
#one bias to the 10 nodes in the hidden layer
#[# nodes, # nodes], sampling from a random uniform distribution from initial_weight01 to initial_weight02 for initial weights

#Second layer
W2 = tf.Variable(tf.random_uniform([1,node_hidden01],initial_weight01,initial_weight02)) 
#second layer: 10 nodes in the first hidden layer to one node in the second
#W2: weights from hidden layer 01 to output layer
b2 = tf.Variable(tf.random_uniform([1],initial_weight01,initial_weight02))


#####
##construct the network
h1 = tf.nn.tanh(tf.matmul(W1,input_ph)+b1)
#y = tanh(x1w11+x2w21)
#tf.matmul: matrix multiplication

output = tf.nn.sigmoid(tf.matmul(W2,h1)+b2) 
#This is the actual construction of the network. 
#When we want to get the output of the network, we will tell tensorflow what to put in the input placeholder, and then we'll run this output function

loss = tf.reduce_sum(tf.square(output - target_ph))
#This is the function we're trying to optimize. The reduce_sum is not really necessary since we only have a single output, just using it to flatten the output.

#optimizer = tf.train.AdamOptimizer(learning_rate) 
optimizer = tf.train.AdamOptimizer(learning_rate)
#This is a fancy version of momentum based gradient descent optimization.

train = optimizer.minimize(loss) 
#This will be how we tell the network to train on an example


In [0]:
def test():
    MSE = 0.0
    for i in xrange(len(x_data)):
      MSE += sess.run(loss,feed_dict={input_ph: x_data[i].reshape([3,1]),target_ph: y_data[i].reshape([1,1])}) #test on a test data point. feed_dict is how you pass things in to the placeholders created above
    MSE /= len(x_data)
    return MSE

def accuracy():
    temp =[] 
    for i in xrange(len(y_data)):
        temp.append(sess.run(output,feed_dict={input_ph: x_data[i].reshape([3,1])}))
    return temp
    
    
def hidden():
    temp = []
    for i in xrange(len(y_data)):
        temp.append(sess.run(h1,feed_dict={input_ph: x_data[i].reshape([3,1])}))
    return temp

In [38]:
inputfeature = numpy.transpose(x_data)
print(inputfeature.shape)
inputlabel = numpy.transpose(y_data)
print(inputlabel.shape)

(3, 52)
(52,)


In [39]:
# Launch the graph -- tell tensorflow to initialize everything.
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init) #first argument to sess.run is function to run, here we're running the initialize function

# Fit the function
print "Pre training MSE:", test
ac = []
hid = []
los = []

for step in xrange(1):
  for item in xrange(nsamples): 
    sess.run(train,feed_dict={input_ph: inputfeature,target_ph: inputlabel}) 
  #sess.run(train): this will train the model, update the weights
  #Run training on an example
  ac.append(accuracy())
  hid.append(hidden())
  los.append(test())
  if step % 100 == 0:
    print "On step %i, test MSE %f" %(step,test())

Pre training MSE: <function test at 0x7fd080e5cde8>


ValueError: ignored